In [62]:
import pandas as pd
import numpy as np

SENTIMENTS = "anger fear joy sadness".split()

def load_predictions(folder_path: str) -> dict:
    result = {}
    for sen in SENTIMENTS:
        path = f"{folder_path}/{sen}-pred.txt"
        result[sen] = pd.read_csv(path, sep='\t', names=['id', 'low', 'medium', 'high'])
        result[sen].set_index("id")
    return result

def get_comparison(a: dict, b: dict, sentiment=None) -> [dict, pd.DataFrame]:
    result = {}
    for sen in SENTIMENTS:
        intensity_a = a[sen].apply(lambda x: np.argmax(x[1:]), axis=1)
        intensity_b = b[sen].apply(lambda x: np.argmax(x[1:]), axis=1)
        different = intensity_a != intensity_b
        result[sen] = pd.DataFrame({"id": a[sen]["id"], "a": intensity_a, "b": intensity_b, "different": different}).set_index("id")
    if sentiment is not None:
        return result[sentiment]
    percentage = {}
    equal_count = {}
    different_count = {}
    for sen in SENTIMENTS:
        df = result[sen]
        equal_count[sen] = df[df["different"] == True]["different"].count()
        different_count[sen] = df[df["different"] == False]["different"].count()
        percentage[sen] = equal_count[sen] / len(df.index)
    meta_result = {"result": result, "different_count": different_count, "equal_count": equal_count, "percentage": percentage}
    return meta_result

def print_percentage(r: dict) -> None:
    for sen in SENTIMENTS:
        print(f"{sen}: {r['percentage'][sen]}")

In [63]:
base_predictions = load_predictions("../assignment_1/predictions")
felipe_benja_predictions = load_predictions("../felipe-benja/predictions")

base_fb_comparison = get_comparison(base_predictions, felipe_benja_predictions)

In [64]:
print_percentage(base_fb_comparison)
get_comparison(base_predictions, felipe_benja_predictions, "anger")

anger: 0.3026315789473684
fear: 0.3326633165829146
joy: 0.35294117647058826
sadness: 0.3536404160475483


,a,b,different
id,,,
10941,1,1,False
10942,0,0,False
10943,1,0,True
10944,1,1,False
10945,1,0,True
...,...,...,...
11696,1,1,False
11697,1,1,False
11698,1,0,True
